In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

In [10]:
##初始化权值
def weight_variable(shape):
    initial= tf.truncated_normal(shape,stddev=0.1) #生成一个阶段的正太分布
    return tf.Variable(initial)
## 初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
## 卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
## 池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')




    

In [11]:
def variable_summaries(var):
    with tf.name_scope('summeries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图

In [12]:
#input 
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])

In [13]:
##改变x的格式转变为4D的向量 [batch,in_height,in_weight,in_channel]
with tf.name_scope('input_reshape'):   
    x_image = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',x_image,10)
    



In [16]:
with tf.name_scope('layer'):
    
    ##初始化第一个卷积层的权重和偏置
    with tf.name_scope('Weight_1'):
        W_conv1 = weight_variable([5,5,1,32]) ## 5x5的采样窗口,32个卷积核从1个平面抽取特征
        variable_summaries(W_conv1)
        
    with tf.name_scope('bias_1'):
        b_conv1 = bias_variable([32]) ##每一个卷积核一个偏执量
        variable_summaries(b_conv1)

##把x_image 和权值向量进行卷积,再加上偏置量,然后应用于relu 激活函数
    with tf.name_scope('h_conv1'):
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
    with tf.name_scope('pooling_1'):
        h_pool1 = max_pool_2x2(h_conv1) ##进行max_pooling
#2layer
## 初始化第二个卷积层的权重和偏置
    with tf.name_scope('weight_2'):
        W_conv2 = weight_variable([5,5,32,64]) ## 5x5的采样窗口,64个卷积核从32个平面抽取特征
        variable_summaries(W_conv2)
    with tf.name_scope('bias_2'):
        b_conv2 = bias_variable([64]) ##每一个卷积核一个偏执量
        variable_summaries(b_conv2)
    with tf.name_scope('h_conv2'):
        ##把h_pool1 和权值向量进行卷积,再加上偏置量,然后应用于relu 激活函数
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
    with tf.name_scope('pooling_2'):
        h_pool2 = max_pool_2x2(h_conv2) ##进行max_pooling
#fully connected
## 28x28 的图片第一次 卷积 后还是28x28,第一次池化后变成 14x14
## 第二次 卷积后 为 14x14,第二次池化后 成了7x7
# 通过上面操作后得到 64张7x7的平面

## 初始化第一个全连接层的权值      
    with tf.name_scope('weight_3'):
        W_fc1 = weight_variable([7*7*64,1024]) # 上一层有了7*7*64个神经元,全连接层有了1024个神经元
        variable_summaries(W_fc1)
    with tf.name_scope('bias_3'):
        b_fc1 = bias_variable([1024]) ## 1024个节点
        variable_summaries(b_fc1)
    with tf.name_scope('h_pool2_flat'):
        ## 把吃话曾2的输出扁平化为1维
        h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
    with tf.name_scope('relu'):
        ## 求第一个全连接层的输出
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
    with tf.name_scope('dropout_1'):
        ## keep_prob 用来表示 神经元 的输出概率
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    with tf.name_scope('weight_4'):
        ## 初始化第二个全连接层
        W_fc2 = weight_variable([1024,10])
        variable_summaries(W_fc2)
    with tf.name_scope('bias_4'):
        b_fc2 = bias_variable([10])
        variable_summaries(b_fc2)
    with tf.name_scope('softmax'):
        ## 计算输出
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
        

    

In [17]:
## 交叉熵代价函数
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',cross_entropy)
## 使用 AdamOptimizer进行优化
#####不同的optimizer造成的结果一般千差万别
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
## 结果存放在一个布尔列表中
with tf.name_scope('accuracy'):
    with tf.name_scope('correct-prediction'):
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)) ## argmax返回一维张量中最大的值所在的位置
## 求准确率
    with tf.name_scope('accuracy'):
        accuracy= tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [18]:
merged = tf.summary.merge_all()

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('logs/mnistCNN',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys =mnist.train.next_batch(batch_size)
            summary,_ =sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        writer.add_summary(summary,epoch)
        train_acc = sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print ("Iter" + str(epoch)+" ,Testing Accuracy" +str(acc)+'training acc='+str(train_acc))

Iter0 ,Testing Accuracy0.9524training acc=0.95
Iter1 ,Testing Accuracy0.9706training acc=0.97
Iter2 ,Testing Accuracy0.9768training acc=0.98
Iter3 ,Testing Accuracy0.9817training acc=0.94
Iter4 ,Testing Accuracy0.9848training acc=0.97
Iter5 ,Testing Accuracy0.9857training acc=0.99
Iter6 ,Testing Accuracy0.9871training acc=1.0
Iter7 ,Testing Accuracy0.9869training acc=1.0
Iter8 ,Testing Accuracy0.9883training acc=1.0
Iter9 ,Testing Accuracy0.9895training acc=0.98
Iter10 ,Testing Accuracy0.9901training acc=1.0
Iter11 ,Testing Accuracy0.9908training acc=1.0
Iter12 ,Testing Accuracy0.9899training acc=0.99
Iter13 ,Testing Accuracy0.9892training acc=1.0
Iter14 ,Testing Accuracy0.9889training acc=0.99
Iter15 ,Testing Accuracy0.9907training acc=0.99
Iter16 ,Testing Accuracy0.9905training acc=0.99
Iter17 ,Testing Accuracy0.9909training acc=1.0
Iter18 ,Testing Accuracy0.9916training acc=1.0
Iter19 ,Testing Accuracy0.991training acc=1.0
Iter20 ,Testing Accuracy0.991training acc=1.0
